In [ ]:
import numpy as np
import pandas as pd
import os
import openpyxl
import tensorflow as tf
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from tensorflow.keras.layers import LSTM,Dense,GRU,Dropout,SimpleRNN
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
n_hours = 5
n_features = 43
train_num_24 = 2789
train_num_25 = 2892
train_num_19 = 1934

In [ ]:
from google.colab import drive
filename = '/content/bigcon_data.xlsx'

df = pd.read_excel(filename)

In [ ]:
# df_metrix 만드는 과정
df_matrix = df.to_numpy()
df_matrix = df_matrix[1:train_num_25, 5:]
print(df_matrix)

df_matrix = df_matrix.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
df_matrix = scaler.fit_transform(df_matrix)

df_matrix

In [ ]:
# print(df_matrix)
lag = series_to_supervised(df_matrix, n_hours,1)

# print(lag)
# print(lag.shape[1])
# for i in range(215,258):
#     lag.drop(lag.columns[[i]], axis = 1, inplace = True)


lag = lag.values

tmp = lag.shape[1]-42

lag = lag[:,0:tmp]
print(lag.shape)

In [ ]:
n_train_hours = train_num_19
train = lag[:n_train_hours, :]
test = lag[n_train_hours:, :]

In [ ]:
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
plt.figure(figsize=(20, 10))
# sns.lineplot(y=df_scaled['유입량'], x=df['일자'])
# plt.xlabel('TIME')
# plt.ylabel('AMOUNT')
plt.plot(df['유입량'])
plt.xlabel('TIME')
plt.ylabel('AMOUNT')

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1],train_X.shape[2]), activation='tanh', return_sequences=False))
model.add(Dense(1))

In [ ]:
import os
from keras import optimizers

optimizer = tf.keras.optimizers.Adam(lr=0.01)

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(train_X, train_y, 
                                    epochs=100, 
                                    batch_size=32,
                                    validation_data=(test_X, test_y), 
                                    callbacks=[early_stop, checkpoint])

In [ ]:
model.load_weights(filename)
pred = model.predict(test_X)

pred.shape

In [ ]:
RMSE = mean_squared_error(test_y,pred)**0.5
RMSE

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_y, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()